In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate
from keras.models import Model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
train_img = os.listdir('train_images')

In [3]:
len(train_img)

2528

In [4]:
train_img[0]

'train_01927.png'

In [5]:
x = cv2.imread('train_images/'+train_img[0])

In [6]:
print (x.shape)

(512, 512, 3)


In [7]:
img_list=[]
for i,img in enumerate(train_img):
    x = cv2.imread('train_images/'+img, cv2.IMREAD_GRAYSCALE)
    img_list.append(x)


In [8]:
train_data = np.array(img_list)
train_data =  train_data[:,:,:,np.newaxis]
print (train_data.shape)

(2528, 512, 512, 1)


In [9]:
df_y = pd.read_csv('train.csv')
df_y['Label'].value_counts()

0    674
5    644
1    492
3    378
4    240
2    100
Name: Label, dtype: int64

In [10]:
train_y =  df_y['Label'].values

In [11]:
onehot_encoder = OneHotEncoder(sparse=False)
train_y = train_y.reshape(len(train_y), 1)
print (train_y.shape)
train_label = onehot_encoder.fit_transform(train_y)


print ('beofre', train_label.shape)
print (df_y['Label'][:10])
print (train_label[:10,:])

train_label = train_label[:,np.newaxis,np.newaxis,:]

print ('after', train_label.shape)

(2528, 1)
beofre (2528, 6)
0    0
1    1
2    1
3    5
4    5
5    5
6    3
7    0
8    3
9    5
Name: Label, dtype: int64
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
after (2528, 1, 1, 6)


In [20]:
train_x,val_x,train_y,val_y = train_test_split(train_data,train_label,test_size=0.2, shuffle=True)
print (train_x.shape)
print (train_y.shape)
print (val_x.shape)
print (val_y.shape)

(2022, 512, 512, 1)
(2022, 1, 1, 6)
(506, 512, 512, 1)
(506, 1, 1, 6)


IMAGE_LENTH = train_data [0]
IMAGE_HEIGHT = train_data [1]
IMAGE_WIDTH = train_data[2]
IMAGE_CHANNEL = train_data[3]
ch = 4

In [13]:
def conv_block(ch, activation= 'relu', padding='same'):       
    return Conv2D(ch,(3,3),activation= activation, padding =padding )


In [22]:
#def build_model_1(IMAGE_HEIGHT=IMAGE_HEIGHT,IMAGE_WIDTH=IMAGE_WIDTH,ch=ch):
def build_model_1(ch=4, category=6):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    inputs = Input((512,512,1))
    inputs_norms = Lambda(lambda x : x/127.5 -1.0)(inputs)
    conv1 =  conv_block(ch)(inputs_norms)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 =  conv_block(ch*4)(pool2)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)

    
    up6 = concatenate([UpSampling2D(size=(2,2))(conv5), conv4], axis=3)
    conv6 = conv_block(ch*8)(up6)
    conv6 = conv_block(ch*8)(conv6)
    
    up7 = concatenate ([UpSampling2D(size=(2,2))(conv6), conv3],  axis=3)
    conv7 = conv_block(ch*4)(up7)
    conv7 = conv_block(ch*4)(conv7)
    
    up8 = concatenate([UpSampling2D(size=(2,2))(conv7), conv2], axis=3)
    conv8 = conv_block(ch*2)(up8)
    conv8 = conv_block(ch*2)(conv8)
    
    up9 = concatenate([UpSampling2D(size=(2,2))(conv8), conv1],  axis=3)
    conv9 = conv_block(ch)(up9)
    conv9 = conv_block(ch)(conv9)
    
    
    
    conv10 = Conv2D(1,(1,1))(conv9)
    
    #model = Model(inputs= inputs, outputs = pool4)

    conv_11 =  conv_block(ch)(conv10)
    conv_11 = conv_block(ch)(conv_11)
    pool_11 = MaxPooling2D(pool_size=(2,2))(conv_11)

    conv_12 =  conv_block(ch*2)(pool_11)
    conv_12 = conv_block(ch*2)(conv_12)
    pool_12 = MaxPooling2D(pool_size=(2,2))(conv_12)

    conv_13 =  conv_block(ch*4)(pool_12)
    conv_13 = conv_block(ch*4)(conv_13)
    pool_13 = MaxPooling2D(pool_size=(2,2))(conv_13)


    conv_14 =  conv_block(ch*8)(pool_13)
    conv_14 = conv_block(ch*8)(conv_14)
    pool_14 = MaxPooling2D(pool_size=(2,2))(conv_14)

    conv_15 =  conv_block(ch*16)(pool_14)
    conv_15 = conv_block(ch*16)(conv_15)
    pool_15 = MaxPooling2D(pool_size=(2,2))(conv_15)

    conv_16 =  conv_block(ch*16)(pool_15)
    conv_16 = conv_block(ch*16)(conv_16)
    pool_16 = MaxPooling2D(pool_size=(2,2))(conv_16)

    conv_17 =  conv_block(ch*16)(pool_16)
    conv_17 = conv_block(ch*16)(conv_17)
    pool_17 = MaxPooling2D(pool_size=(2,2))(conv_17)

    conv_18 =  conv_block(ch*16)(pool_17)
    conv_18 = conv_block(ch*16)(conv_18)
    pool_18 = MaxPooling2D(pool_size=(2,2))(conv_18)

    conv_19 =  conv_block(ch*8)(pool_18)
    conv_19 = conv_block(ch*8)(conv_19)
    pool_19 = MaxPooling2D(pool_size=(2,2), padding='valid')(conv_19)


    conv_20 = Conv2D(category,(1,1),activation='softmax')(pool_19)

  
    
  
    model = Model(inputs= inputs, outputs=conv_20)
    
    return model
    
    

def build_model_2(build_model_1):
    #inputs = Input((IMAGE_HEIGHT,IMAGE_WIDTH,ch))
    conv = build_model_1()
    conv1 =  conv_block(ch)(conv)
    conv1 = conv_block(ch)(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 =  conv_block(ch*2)(pool1)
    conv2 = conv_block(ch*2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 =  conv_block(ch*4)(pool2)
    conv3 = conv_block(ch*4)(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    
    conv4 =  conv_block(ch*8)(pool3)
    conv4 = conv_block(ch*8)(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    conv5 =  conv_block(ch*16)(pool4)
    conv5 = conv_block(ch*16)(conv5)
    pool5 = MaxPooling2D(pool_size=(2,2))(conv5)

    conv6 =  conv_block(ch*16)(pool5)
    conv6 = conv_block(ch*16)(conv6)
    pool6 = MaxPooling2D(pool_size=(2,2))(conv6)
    
    conv7 =  conv_block(ch*16)(pool6)
    conv7 = conv_block(ch*16)(conv7)
    pool7 = MaxPooling2D(pool_size=(2,2))(conv7)
    
    conv8 =  conv_block(ch*16)(pool7)
    conv8 = conv_block(ch*16)(conv8)
    pool8 = MaxPooling2D(pool_size=(2,2))(conv8)
    
    conv9 =  conv_block(ch*8)(pool8)
    conv9 = conv_block(ch*8)(conv9)
    pool9 = MaxPooling2D(pool_size=(2,2), padding='valid')(conv9)
    

    conv10 = Conv2D(5,(1,1),activation='softmax')(pool9)
    
    
    return conv10

In [23]:
model = build_model_1()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 512, 512, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 512, 512, 4)  40          lambda_2[0][0]                   
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 512, 512, 4)  148         conv2d_39[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [24]:
model.compile (optimizer='Adam', loss = 'categorical_crossentropy')

In [ ]:
model.fit(x=train_x, y=train_y, batch_size=8, epochs=100, verbose=1, validation_split=0.2,  shuffle=True)

Train on 1617 samples, validate on 405 samples
Epoch 1/100
1617/1617 [==============================] - 3098s 2s/step - loss: 1.6698 - val_loss: 1.6784
Epoch 2/100
1616/1617 [============================>.] - ETA: 1s - loss: 1.6578

with open('train_data.pickle', 'wb') as f:
    pickle.dump(train_data, f)

with open('train_data.pickle', 'rb') as f:
    train_data = pickle.load(f)

print (train_data.shape)    
